In [183]:
import subprocess
import sqlite3
import os
import joblib
import re
import unicodedata
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer, models

word_embedding_model = models.Transformer('neuralmind/bert-base-portuguese-cased')
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])


In [199]:
def carregar_textos(db_path="projeto_fernando_pessoa/textos.db"):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.execute("SELECT autor, texto FROM textos")
    rows = cursor.fetchall()
    conn.close()
    textos_por_autor = {}
    for autor, texto in rows:
        textos_por_autor.setdefault(autor, []).append(texto)
    return textos_por_autor

In [200]:
def limpar_texto(texto):
    # Remove múltiplos espaços
    texto = re.sub(r'\s+', ' ', texto)
    # Remove caracteres especiais
    texto = re.sub(r'[^a-zA-Z0-9À-ú,.!?;:\-\s]', '', texto)
    # Normaliza acentos
    texto = unicodedata.normalize('NFKC', texto)
    # Strip
    return texto.strip()

In [208]:
def gerar_pkls_autores(textos_por_autor, pasta_modelos='modelos_autores', treino_ratio=0.8):
    os.makedirs(pasta_modelos, exist_ok=True)

    todos_textos = []
    for autor, textos in textos_por_autor.items():
        textos_limpos = [limpar_texto(t) for t in textos]
        textos_treino, textos_val = train_test_split(textos_limpos, train_size=treino_ratio, random_state=42)

        # PKL individual
        joblib.dump({'autor': autor, 'textos': textos_treino}, os.path.join(pasta_modelos, f"{autor}_treino.pkl"))
        joblib.dump({'autor': autor, 'textos': textos_val}, os.path.join(pasta_modelos, f"{autor}_validacao.pkl"))
        print(f"PKLs criados para {autor}: treino({len(textos_treino)}) / validação({len(textos_val)})")

        # Adiciona ao PKL geral
        for t in textos_treino:
            todos_textos.append({'autor': autor, 'texto': t})

    # PKL geral com todos os textos de treino
    joblib.dump(todos_textos, os.path.join(pasta_modelos, 'todos_autores_treino.pkl'))
    print(f"PKL geral com todos os autores criado: {len(todos_textos)} textos")



In [210]:
def treinar_embeddings(model_name, pasta_modelos='modelos_autores'):
    model = SentenceTransformer(model_name)
    for arquivo in os.listdir(pasta_modelos):
        if '_treino.pkl' in arquivo:
            dados = joblib.load(os.path.join(pasta_modelos, arquivo))
            embeddings = model.encode(dados['textos'], convert_to_numpy=True)
            joblib.dump({'autor': dados['autor'], 'textos': dados['textos'], 'embeddings': embeddings},
                        os.path.join(pasta_modelos, f"{dados['autor']}_{model_name.replace('/', '-')}_treino.pkl"))
    print(f"Treino concluído para modelo {model_name}")

In [212]:
def treinar_embeddings_geral(model_name, pasta_modelos='modelos_autores'):
    model = SentenceTransformer(model_name)
    pkl_geral = os.path.join(pasta_modelos, 'todos_autores_treino.pkl')

    if os.path.exists(pkl_geral):
        todos_textos = joblib.load(pkl_geral)
        textos = [t['texto'] for t in todos_textos]
        autores = [t['autor'] for t in todos_textos]

        embeddings = model.encode(textos, convert_to_numpy=True)
        joblib.dump({'autores': autores, 'textos': textos, 'embeddings': embeddings},
                    os.path.join(pasta_modelos, f'todos_autores_{model_name.replace("/", "-")}_treino.pkl'))
        print(f"Embeddings gerais treinados para {model_name} com {len(textos)} textos")


In [213]:
def classificar_texto_geral(texto, modelo_embedding, pasta_modelos='modelos_autores'):
    model = SentenceTransformer(modelo_embedding)
    embedding_texto = model.encode([texto], convert_to_numpy=True)

    # Carrega embeddings do PKL geral
    pkl_geral = os.path.join(pasta_modelos, f'todos_autores_{modelo_embedding.replace("/", "-")}_treino.pkl')
    dados = joblib.load(pkl_geral)

    sim = cosine_similarity(embedding_texto, dados['embeddings']).flatten()
    autor_predito = dados['autores'][sim.argmax()]

    # Calcula similaridade média por autor (opcional)
    resultados = {}
    for a, s in zip(dados['autores'], sim):
        resultados[a] = max(resultados.get(a, 0), s)

    resultados = sorted(resultados.items(), key=lambda x: x[1], reverse=True)

    return autor_predito, resultados


In [214]:
def avaliar_modelo_completo(model_name, pasta_modelos='modelos_autores'):

    import numpy as np
    from sentence_transformers import SentenceTransformer
    from sklearn.metrics.pairwise import cosine_similarity
    import os
    import joblib

    model = SentenceTransformer(model_name)

    embeddings_por_autor = {}
    textos_por_autor = {}

    for arquivo in os.listdir(pasta_modelos):
        if '_treino.pkl' in arquivo and 'todos_autores' not in arquivo:
            dados = joblib.load(os.path.join(pasta_modelos, arquivo))
            autor = dados['autor']
            textos = dados['textos']
            textos_por_autor[autor] = textos_por_autor.get(autor, []) + textos

            embeddings = model.encode(textos, convert_to_numpy=True)
            if autor in embeddings_por_autor:
                embeddings_por_autor[autor] = np.vstack([embeddings_por_autor[autor], embeddings])
            else:
                embeddings_por_autor[autor] = embeddings

    # Classificação e contagem de acertos
    acertos = 0
    total = 0

    for autor_real, textos in textos_por_autor.items():
        for texto in textos:
            texto_embed = model.encode([texto], convert_to_numpy=True)
            similares = [(autor, cosine_similarity(texto_embed, emb).mean())
                         for autor, emb in embeddings_por_autor.items()]
            autor_predito = max(similares, key=lambda x: x[1])[0]
            if autor_predito == autor_real:
                acertos += 1
            total += 1

    acuracia = acertos / total if total > 0 else 0
    print(f"Acurácia do modelo {model_name}: {acuracia * 100:.2f}%")
    return acuracia

In [215]:
def selecionar_melhor_modelo_completo(modelos, pasta_modelos='modelos_autores'):
    resultados = {}
    for modelo in modelos:
        print(f"\nAvaliando modelo: {modelo}")
        acc = avaliar_modelo_completo(modelo, pasta_modelos)
        resultados[modelo] = acc

    melhor_modelo = max(resultados, key=resultados.get)
    print(f"\nMelhor modelo: {melhor_modelo} com acurácia {resultados[melhor_modelo] * 100:.2f}%")
    return melhor_modelo

In [ ]:
# Garantir que PKLs já foram criados
textos_por_autor = carregar_textos("projeto_fernando_pessoa/textos.db")
gerar_pkls_autores(textos_por_autor)

modelos_teste = [
    # 'paraphrase-multilingual-MiniLM-L12-v2',
    # 'all-MiniLM-L6-v2',
    # 'distiluse-base-multilingual-cased-v2',
    # 'neuralmind/bert-base-portuguese-cased',
    'neuralmind/bert-large-portuguese-cased',
    'all-MiniLM-L12-v2',
    'paraphrase-multilingual-mpnet-base-v2'
]

melhor_modelo = selecionar_melhor_modelo_completo(modelos_teste)

PKLs criados para Ricardo Reis: treino(214) / validação(54)
PKLs criados para Bernardo Soares: treino(325) / validação(82)
PKLs criados para Álvaro de Campos: treino(302) / validação(76)
PKLs criados para Alberto Caeiro: treino(101) / validação(26)
PKL geral com todos os autores criado: 942 textos

Avaliando modelo: paraphrase-multilingual-MiniLM-L12-v2
Acurácia do modelo paraphrase-multilingual-MiniLM-L12-v2: 69.53%

Avaliando modelo: all-MiniLM-L6-v2
Acurácia do modelo all-MiniLM-L6-v2: 47.35%

Avaliando modelo: distiluse-base-multilingual-cased-v2


In [ ]:
def classificar_texto(texto, melhor_modelo, pasta_modelos='modelos_autores'):
    """
    Classifica um texto usando embeddings do melhor modelo.
    Retorna o autor mais provável e a similaridade com todos os autores.
    """
    from sentence_transformers import SentenceTransformer
    from sklearn.metrics.pairwise import cosine_similarity
    import joblib
    import os
    import numpy as np

    model = SentenceTransformer(melhor_modelo)
    embedding_texto = model.encode([texto], convert_to_numpy=True)

    # Carregar embeddings de todos os autores gerados com o melhor modelo
    modelo_nome_arquivo = melhor_modelo.replace('/', '-')
    pkls_modelo = [f for f in os.listdir(pasta_modelos) if modelo_nome_arquivo in f and '_treino.pkl' in f]

    resultados = []
    for arquivo in pkls_modelo:
        dados = joblib.load(os.path.join(pasta_modelos, arquivo))
        sim = cosine_similarity(embedding_texto, dados['embeddings']).mean()
        resultados.append((dados['autor'], sim))

    resultados.sort(key=lambda x: x[1], reverse=True)
    autor_predito = resultados[0][0] if resultados else None

    return autor_predito, resultados


In [112]:
def classificar_autor_com_similaridade(texto, pasta_modelos='modelos_autores', modelo_embedding='distiluse-base-multilingual-cased-v2'):
    from sentence_transformers import SentenceTransformer
    from sklearn.metrics.pairwise import cosine_similarity
    import joblib
    import os

    model = SentenceTransformer(modelo_embedding)
    embedding_texto = model.encode([texto], convert_to_numpy=True)

    resultados = []

    # Filtra apenas os PKLs que foram gerados com o mesmo modelo
    modelo_nome_arquivo = modelo_embedding.replace('/', '-')
    pkls_modelo = [f for f in os.listdir(pasta_modelos) if modelo_nome_arquivo in f and '_treino.pkl' in f]

    for arquivo in pkls_modelo:
        dados = joblib.load(os.path.join(pasta_modelos, arquivo))
        sim = cosine_similarity(embedding_texto, dados['embeddings'])
        sim_media = sim.mean()
        resultados.append((dados['autor'], sim_media))

    resultados.sort(key=lambda x: x[1], reverse=True)
    autor_predito = resultados[0][0] if resultados else None
    return autor_predito, resultados


In [108]:
def interpretar_texto_ollama(texto_a_interpretar, autor_predito, pasta_modelos='modelos_autores', modelo_llm='phi3'):
    """
    Interpreta um texto de acordo com o estilo do autor usando o Ollama CLI local.
    """
    try:
        dados = joblib.load(os.path.join(pasta_modelos, f"{autor_predito}.pkl"))
        textos_exemplo = dados.get('textos', [])
        exemplo_texto = textos_exemplo[0][:500] if textos_exemplo else "Nenhum exemplo de texto disponível para este autor."
    except FileNotFoundError:
        exemplo_texto = "Nenhum exemplo de texto disponível para este autor."
        print(f"Aviso: Arquivo de modelo para o autor '{autor_predito}' não encontrado.")

    prompt = f"""
Você é um especialista em escrita e poesia, com profundo conhecimento sobre o estilo do autor {autor_predito}.
O texto a seguir foi escrito por outra pessoa: "{texto_a_interpretar}"
Interprete este texto de forma criativa, reescrevendo-o completamente para que ele pareça ter sido escrito
pelo autor {autor_predito}. Use a voz, as metáforas e o estilo de escrita característicos dele.
Aqui está um exemplo do estilo de escrita do autor: "{exemplo_texto}"
"""

    try:
        result = subprocess.run(
            ["ollama", "run", modelo_llm],
            input=prompt,
            capture_output=True,
            text=True,
            encoding='utf-8'
        )
    except FileNotFoundError:
        print("Erro: O comando 'ollama' não foi encontrado. Certifique-se de que o Ollama está instalado e no seu PATH.")
        return None

    if result.returncode != 0:
        print(f"\n--- Erro do Ollama ---")
        print(f"Código de retorno: {result.returncode}")
        print(f"Saída de erro: {result.stderr}")
        raise RuntimeError(f"Erro ao chamar Ollama: {result.stderr}")

    if result.stdout is None:
        return ""

    return result.stdout.strip()


In [116]:
texto_teste = """Mover-se é viver, dizer-se é sobreviver. Não há nada de real na vida que o não seja porque se descreveu bem. Os críticos da casa pequena soem apontar que tal poema, longamente ritmado, não quer, afinal, dizer senão que o dia está bom."""

# autor_predito, todas_similaridades = classificar_autor_com_similaridade(texto_teste)
autor_predito, todas_similaridades = classificar_texto(texto_teste, melhor_modelo)
# autor_predito, todas_similaridades = classificar_texto_geral(texto_teste, melhor_modelo)



print(f"Autor mais provável: {autor_predito}\n")
print("Similaridade com cada autor:")
for autor, sim in todas_similaridades:
    print(f"{autor}: {sim:.2f}")


# interpretacao = interpretar_texto_ollama(texto_teste, autor_predito)
# print(f"\n--- Interpretação pelo LLM ---\n{interpretacao}")

Autor mais provável: Alberto Caeiro

Similaridade com cada autor:
Alberto Caeiro: 0.27
Ricardo Reis: 0.26
Álvaro de Campos: 0.25
Bernardo Soares: 0.13
